In [52]:
import pandas as pd
import numpy as np
import os
import boto3

from dotenv import find_dotenv, load_dotenv
from pathlib import Path

env_file = find_dotenv()
load_dotenv(env_file)

AWS_SECRECT_KEY = os.getenv("AWS_SECRET_KEY")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
BUCKET_NAME_PARTS = os.getenv('AWS_BUCKET_NAME_PARTS')
AWS_BUCKET_NAME_ATTACHMENTS = os.getenv('AWS_BUCKET_NAME_ATTACHMENTS')

In [26]:
df = pd.read_csv("./item_data.csv", index_col=0)

In [28]:
df_step = df[df["file_key"].str.lower().str.endswith(("step", "stp"))]

In [29]:
df_step.head()

,item_id,file_name,file_key,quantity,technology,technology_id,area,depth,width,height,volume
0,69234,HulkeyV11.05_S0.5_H3.stp,9dd88fce-ba13-4034-a31a-e184f13c6f4d/HulkeyV11...,100,LPBF,8a705112-174c-4419-bcfb-ab2716607a8c,2202.591709,7.870000,9.000972,52.968502,872.438624
2,69440,Z008138_001_FPT01_INJECTION_MOLDED_HUB___BOTTO...,279cb696-d9e1-4d9e-a119-e0b7c625b6ce/Z008138_0...,12,LPBF,8a705112-174c-4419-bcfb-ab2716607a8c,45176.734297,183.593580,52.719967,104.887495,197126.040695
4,71814,Injector_seat.stp,eaa2850c-5cb4-4fe2-aa43-d554cf6bb3e3/Injector_...,1,SLA (Resin),9f9c45e6-2d02-4ba6-893b-c836d27b7343,21359.757786,47.000000,88.491932,56.500000,44846.286062
5,71973,POS10_10_443_403.step,5cbc2edc-d846-4d55-b948-508cfcd6e4cc/POS10_10_...,1,CNC,feb0f26f-94a5-4be2-9d40-761bb2857ab6,17812.326994,25.000000,110.000000,30.000000,51984.257381
6,104757,MakerVerse_Sample.step,28670b62-0bf9-48b0-8407-321583bb665a/MakerVers...,1,CNC,9c5d6810-29d6-48c2-9180-a2a830b7be62,208702.015625,163.662476,185.610687,163.662476,963533.500000


In [40]:
test_df = df[["item_id", "file_key"]][:10]

In [54]:
def download_s3_file():
    s3 = boto3.client(
        "s3",
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRECT_KEY,
    )
    bucket_name = "refined-zone"
    file_key = "hubspot/companies/history/hubspot_company_trimmed_data_history_00000_00999.json"
    file_path = "test.json"
    s3.download_file(bucket_name, file_key, file_path)
download_s3_file()

ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden

In [ ]:
def download_part_files(part_list):
    s3 = boto3.client("s3")
    with tqdm(part_list) as pbar:
        for file_id, file_key in pbar:
            pbar.set_description(f"File name: {file_key}")
            folder = Path("./part_files")
            folder.mkdir(exist_ok=True)
            file_path = folder / file_key
            s3.download_file(BUCKET_NAME_PARTS, file_key, file_path)
        

In [50]:
sts = boto3.client("sts")
print(sts.get_caller_identity())

NoCredentialsError: Unable to locate credentials